<a href="https://colab.research.google.com/github/TheCaduceus/Multi-Cloud-Transfer-Tool/blob/main/Rclone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Connect Google Account:**

**First connect Google Drive, to enable MCT Tool to save/retrive its configuration files.**

<b>Options:</b><br>
`FORCE_REMOUNT` :<b> Forcefully remount Google Drive.</b><br>
`CUSTOM_MOUNT_POINT` :<b> Choose different path for mounting Google Drive.</b><br>
`DEBUG_MODE` :<b> To enable or disable DEBUG Mode.</b>

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Google-Drive-Logo.png" height="50" alt="Google-Drive-Logo"/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>

from pathlib import Path
from contextlib import suppress
from sys import exit as end
from google.colab import output
from google.colab.drive import mount, flush_and_unmount as unmount

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
FORCE_REMOUNT = True #@param {type:"boolean"}
CUSTOM_MOUNT_POINT = False #@param {type:"boolean"}
DEBUG_MODE = False #@param {type:"boolean"}

if CUSTOM_MOUNT_POINT:
  Custom_Mount_Path = input('Please enter Custom Mount Path:\n')
  if not Custom_Mount_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Mount_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Mount_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
else:
  Custom_Mount_Path = '/content/drive'

mount._DEBUG = DEBUG_MODE

if MODE == "MOUNT":
  print("Please Wait! Mounting...")
  mount(Custom_Mount_Path, force_remount= FORCE_REMOUNT)
  if not DEBUG_MODE:
    output.clear()
  print(f'Mounted at {Custom_Mount_Path}')
elif MODE == 'UNMOUNT':
  with suppress(ValueError):
    print('Please Wait! UNMOUNT in progress...')
    unmount()
  get_ipython().system_raw('rm -rf /root/.config/Google/DriveFS')

####**Setup & Configure:**

**Download setup files & configure the environment:**

<b>Options:</b><br>
`INSTALL_RCLONE_FLAVOR`<b>: Switch Rclone's release branch from stable to beta or vice-versa.</b><br>
`FORCE_SETUP`<b>: Forcefully install Rclone by excluding all other dependencies and download them separately. Useful if any dependency (except Rclone) having temporary setup problem.</b><br>
`CUSTOM_SH_FILE`:<b> Get custom .sh file from external source (URL or File Path) and execute it to install additional dependecies or requirements with or without normal setup.</b>

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Setup_img.png" height="50" alt="Setup-Logo"/></center>
#@markdown <center><h3><b>Setup & Configure Environment</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from time import sleep
from requests import get as get_url
from urllib.parse import urlparse as validate_url
from google.colab import output
from json import dumps as dump_json

current_version = "v5.1.0"
do_normal_setup = "1"

# Check for latest version
try:
	json_data = get_url("https://assets.thecaduceus.eu.org/MCT/json/mct.json").json()
	print("Connected with CDN.")
	if current_version == json_data["version"]:
		print(f'Great Job! you are using latest version: {json_data["version"]}')
	else:
		print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]}, use latest notebook available here:\n{json_data["notebookLink"]}')
		sleep(10)
except Exception as error_code:
	end(f'Failed to connect with CDN!\nTraceback:\n{error_code}')

# Save json data locally
with open('mct.json', 'w+') as mct_json:
	mct_json.write(dump_json(json_data))

# Flavor Flags
Stable = ""
Beta = "-s beta"

# User Input
INSTALL_RCLONE_FLAVOR = Stable #@param ["Stable", "Beta"] {type:"raw"}
FORCE_SETUP = False #@param {type:"boolean"}
CUSTOM_SH_FILE = False #@param {type:"boolean"}

if CUSTOM_SH_FILE:

	Custom_sh_Source = input("Provide the Custom Install.sh file source, it can be URL or File Path:\n")
	use_sudo = input("Do you want to execute the .sh file as sudo? 1 = Yes and 0 = No:\n")

	if use_sudo not in ['0','1']:
		end("Unable to check for latest version!")
	elif not Custom_sh_Source:
		end("Custom sh file source can't be blank!")
	elif all([validate_url(Custom_sh_Source).scheme, validate_url(Custom_sh_Source).netloc]):
		try:
			get_url(Custom_sh_Source)
			if use_sudo == '1':
				!curl {Custom_sh_Source} | sudo bash
				print('Execution completed as sudo!')
			elif use_sudo == '0':
				!curl {Custom_sh_Source} | bash
				print('Execution completed!')
		except ConnectionError:
			end("Unable to connect with given sh file URL.")
		except Exception as error_code:
			end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")
	elif not Path(Custom_sh_Source).exists():
		end("Provided Custom sh file path not exist!")
	elif not Path(Custom_sh_Source).is_file():
		end("Provided Custom sh file path should of a particular file not directory/folder.")
	elif not PurePath(Custom_sh_Source).suffix == '.sh':
		end("File format should be .sh")
	else:
		try:
			if use_sudo == '1':
				!sudo bash "$Custom_sh_Source"
				print("Execution completed as sudo.")
			else:
				!bash "$Custom_sh_Source"
				print("Execution complete.")
		except Exception as error_code:
			end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")


	# Ask for Normal Setup
	do_normal_setup = input('Do you want to run Normal Setup also? 1 = Yes and 0 = No:\n')

	if do_normal_setup not in ['0','1']:
		end("Input Error: Value should be 1 or 0")

# Force Setup
if FORCE_SETUP:
	if Path("/content/setup.sh").exists():
		!rm {"/content/setup.sh"}
	print("Starting Force Setup...")
	sleep(4)
	!curl "https://rclone.org/install.sh" | sudo bash {INSTALL_RCLONE_FLAVOR}

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
	try:
		get_url(json_data["setupFile"])
		!curl {json_data["setupFile"]} | sudo bash {INSTALL_RCLONE_FLAVOR}
		!curl {json_data["setupFile"]} > "/content/setup.sh"
		output.clear()
		print("Setup completed!")
	except ConnectionError:
		print('Unable to get setup file!')
	except Exception as error_code:
		print(f'Setup instructions seems to be having mistake(s)!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
	print("Setup file exist and configured already!")

####**Retrive Config Files:**

<b>Options:</b><br>
<b>By default, MCT Tool uses MCT-Config folder located in My Drive to restore configurations.</b><br>
`CUSTOM_IMPORT_PATH` :<b> Scan a custom path (or directory) give by user for usable MCT Configuration files.</b><br>
`CUSTOM_EXPORT_PATH` :<b> Export all your MCT configuration files to a gives path.</b>

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center>
#@markdown <center><h3><b>Fetch Previous MCT Config from My Drive</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')

#@markdown ---
CUSTOM_IMPORT_PATH = False #@param {type:"boolean"}
CUSTOM_EXPORT_PATH = False #@param {type:"boolean"}

# Import Configuration(s)

if CUSTOM_IMPORT_PATH:
  Custom_Path = input('Enter the directory path from where configuration file(s) needs to be imported:\n')
  # Check Path
  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
  elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
    !rclone mkdir "$DRIVE_CONFIG_DIR"
    print('Created: MCT-Config [folder]')

  Custom_Path = PurePath(Custom_Path) # Remove any extra '/'

  # Paths
  IMPORT_TO = DRIVE_CONFIG_DIR
  IMPORT_RCLONE_CONF = f"{Custom_Path}/rclone.conf"
  IMPORT_ACCOUNTS_ZIP = f"{Custom_Path}/accounts.zip"
  IMPORT_ACCOUNTS_FOLDER = f"{Custom_Path}/accounts/"
  ZIP_ACCOUNTS_FOLDER = f"{CMP}/MyDrive/MCT-Config/accounts" # accounts is name of zip, zip file will be saved in MCT-Config
  IMPORT_MEGA_JSON = f"{Custom_Path}/mega.json"
  IMPORT_GOFILE_JSON = f"{Custom_Path}/gofile.json"
  IMPORT_TELEGRAM_JSON = f"{Custom_Path}/telegram.json"
  IMPORT_SESSION_FILES = list(Path(Custom_Path).glob("*.session"))
  # Check 'rclone.conf' file

  if Path(IMPORT_RCLONE_CONF).exists():

    is_import = input('Found: rclone.conf [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_RCLONE_CONF" "$IMPORT_TO"
      print('Successfully imported "rclone.conf" file! (1)')
    else:
      print('Not Found: rclone.conf [file]')

  # Check 'Service Accounts' file/folder

  if Path(IMPORT_ACCOUNTS_ZIP).exists() and Path(IMPORT_ACCOUNTS_FOLDER).exists():

    choice = input('Found both! "accounts.zip" [file] & "accounts" [folder]\n Select one: 0 = accounts.zip [file] or 1 = accounts [folder] or 2 = Ignore\n')

    if choice not in ['0','1','2']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif choice == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
    elif choice == '0':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_ZIP).exists():

    is_import = input('Found: accounts.zip [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_FOLDER).exists():

    is_import = input('Found: accounts [folder] | Do you want to import ? 0=No & 1=Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
  else:
    print('Not Found: accounts.zip [file] or accounts [folder]')

  # Check 'mega.json' file

  if Path(IMPORT_MEGA_JSON).exists():

    is_import = input('Found: mega.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_MEGA_JSON" "$IMPORT_TO"
      print('Successfully imported "mega.json" file! (1)')

  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(IMPORT_GOFILE_JSON).exists():

    is_import = input('Found: gofile.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_GOFILE_JSON" "$IMPORT_TO"
      print('Successfully imported "gofile.json" file! (1)')

  else:
    print('Not Found: gofile.json [file]')

  # Check 'telegram.json' file

  if Path(IMPORT_TELEGRAM_JSON).exists():

    is_import = input('Found: telegram.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_TELEGRAM_JSON" "$IMPORT_TO"
      print('Successfully imported "telegram.json" file! (1)')

  else:
    print('Not Found: telegram.json [file]')

  # Check '*.session' files

  if len(IMPORT_SESSION_FILES) != 0:

    is_import = input('Found: Telegram session files | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      for IMPORT_SESSION_FILE in IMPORT_SESSION_FILES:
        !rclone copy "$IMPORT_SESSION_FILE" "$IMPORT_TO"
      print("Successfully imported telegram session files! (1)")

  else:
    print('Not Found: .session [file]')

# Restore Configuration(s) from MCT-Config folder

if not Path(DRIVE_CONFIG_DIR).exists():
  end('Not Found: MCT-Config [folder]')
else:
  print('Found: MCT-Config [folder]')

# Paths
ROOT = "/content/"
DRIVE_RCLONE_CONF = f"{CMP}/MyDrive/MCT-Config/rclone.conf"
ROOT_RCLONE_CONF = "/root/.config/rclone/"
DRIVE_ACCOUNTS_ZIP = f"{CMP}/MyDrive/MCT-Config/accounts.zip"
DRIVE_MEGA_JSON = f"{CMP}/MyDrive/MCT-Config/mega.json"
DRIVE_GOFILE_JSON = f"{CMP}/MyDrive/MCT-Config/gofile.json"
DRIVE_TELEGRAM_JSON = f"{CMP}/MyDrive/MCT-Config/telegram.json"
DRIVE_SESSION_FILES = list(Path(f"{CMP}/MyDrive/MCT-Config/").glob("*.session"))

# Check 'rclone.conf' file

if Path(DRIVE_RCLONE_CONF).exists():
  !rclone copy "$DRIVE_RCLONE_CONF" "$ROOT_RCLONE_CONF"
  print('Restored: rclone.conf [file]')
else:
  print('Not Found: rclone.conf [file]')

# Check 'Service Accounts' file

if Path(DRIVE_ACCOUNTS_ZIP).exists():
  !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$ROOT"
  print('Restored: accounts.zip [file]')
else:
  print('Not Found: accounts.zip [file]')

# Check 'mega.json' file

if Path(DRIVE_MEGA_JSON).exists():
  !rclone copy "$DRIVE_MEGA_JSON" "$ROOT"
  print('Restored: mega.json [file]')
else:
  print('Not Found: mega.json [file]')

# Check 'gofile.json' file

if Path(DRIVE_GOFILE_JSON).exists():
  !rclone copy "$DRIVE_GOFILE_JSON" "$ROOT"
  print('Restored: gofile.json [file]')
else:
  print('Not Found: gofile.json [file]')

# Check 'telegram.json' file

if Path(DRIVE_TELEGRAM_JSON).exists():
  !rclone copy "$DRIVE_TELEGRAM_JSON" "$ROOT"
  print('Restored: telegram.json [file]')
else:
  print('Not Found: telegram.json [file]')

# Check '*.session' files

if len(DRIVE_SESSION_FILES) != 0:
  for DRIVE_SESSION_FILE in DRIVE_SESSION_FILES:
    !rclone copy "$DRIVE_SESSION_FILE" "$ROOT"
  print('Restored: *.session [files]')
else:
  print('Not Found: *.session [files]')

if CUSTOM_EXPORT_PATH:

  #Paths
  EXPORT_TO = f"{Custom_Path}/MCT-Config/"

  Custom_Path = input('Enter the directory path to where configuration file(s) needs to be exported:\n')

  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Path not found!")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Path is not a valid directory.")
  elif not Path(DRIVE_CONFIG_DIR):
    !rclone mkdir "$DRIVE_CONFIG_DIR"

  # Check 'rclone.conf' file

  if Path(DRIVE_RCLONE_CONF).exists():

    is_export = input('Found: rclone.conf [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_RCLONE_CONF" "$EXPORT_TO"
      print('Export: rclone.conf [file] (1)')
  else:
    print('Not Found: rclone.conf [file]')

    # Check 'Service Accounts' file

  if Path(DRIVE_ACCOUNTS_ZIP).exists():

    is_export = input('Found accounts.zip [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$EXPORT_TO"
      print('Export: accounts.zip [file] (1)')
  else:
    print('Not Found: accounts.zip [file]')

  # Check 'mega.json' file

  if Path(DRIVE_MEGA_JSON).exists():

    is_export = input('Found mega.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_MEGA_JSON" "$EXPORT_TO"
      print('Export: mega.json [file] (1)')
  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(DRIVE_GOFILE_JSON).exists():

    is_export = input('Found gofile.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0', '1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_GOFILE_JSON" "$EXPORT_TO"
      print('Export: gofile.json [file] (1)')
  else:
    print('Not Found: gofile.json [file]')

  # Check 'telegram.json' file

  if Path(DRIVE_TELEGRAM_JSON).exists():

    is_export = input('Found telegram.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_TELEGRAM_JSON" "$EXPORT_TO"
      print('Export: telegram.json [file] (1)')
  else:
    print('Not Found: telegram.json [file]')

  if len(DRIVE_SESSION_FILES) != 0:

    is_export = input('Found Telegram session files | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      for DRIVE_SESSION_FILE in DRIVE_SESSION_FILES:
        !rclone copy "$DRIVE_SESSION_FILE" "$EXPORT_TO"
      print("Export: Telegram session files (1)")
  else:
    print("Not Found: Telegram session [files]")

####**Service Accounts:**

**Options:**<br>
``UPLOAD_SERVICE_ACCOUNTS`` **: Upload new Service Accounts.**<br>
``SAVE_TO_MYDRIVE`` **: Save new/existing Service Accounts to Google Drive.**

In [ ]:
# @markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/google-cloud-logo.png" height="150" alt="Google-Cloud-Logo"/></center>
# @markdown <center><h3><b>Using Service Accounts</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from google.colab import files

# Where user mounted Google Drive?
try:
	CMP = PurePath(Custom_Mount_Path)
except NameError:
	CMP = '/content/drive'

# Paths
ROOT = "/content/"
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"
ROOT_ACCOUNTS_ZIP = "/content/accounts.zip"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
	end('Please download Setup first!')
elif not Path(CMP).exists():
	end('Not Found: Google Drive is not connected!')
elif not Path(DRIVE_CONFIG_DIR).exists():
	!rclone mkdir "$DRIVE_CONFIG_DIR"
	print('Created: MCT-Config [folder] -> MyDrive')

# Configure Options
UPLOAD_SERVICE_ACCOUNTS = True  # @param {type:"boolean"}
SAVE_TO_MYDRIVE = True  # @param {type:"boolean"}

if UPLOAD_SERVICE_ACCOUNTS and SAVE_TO_MYDRIVE:
	print('Upload accounts.zip:\n')
	file = files.upload()
	if list(file.keys())[0] == 'accounts.zip':
		!rclone copy "$ROOT_ACCOUNTS_ZIP" "$DRIVE_CONFIG_DIR"
		!unzip "$ROOT_ACCOUNTS_ZIP" -d "$ROOT"
		print('Uploaded & Configured Service Accounts! (2)')
	else:
		end('File Error: File is not accounts.zip')

elif not UPLOAD_SERVICE_ACCOUNTS and not SAVE_TO_MYDRIVE:
	print('Task Error: No Task to perform :)')

elif UPLOAD_SERVICE_ACCOUNTS:
	print('Upload accounts.zip:\n')
	file = files.upload()
	if list(file.keys())[0] == 'accounts.zip':
		!unzip "$ROOT_ACCOUNTS_ZIP" -d "$ROOT"
		print('Uploaded & Configured Service Accounts! (1)')
	else:
		end('File Error: File is not accounts.zip')

elif Path(ROOT_ACCOUNTS_ZIP).exists():
	print('Found: accounts.zip [file]')
	!unzip "$ROOT_ACCOUNTS_ZIP" -d "$ROOT"
	print('Configured Service Accounts!')

else:
	print('Not Found: accounts.zip [file]')

####**Rclone Operations:**

**Create/Edit/View/Delete your Rclone Config file:<br>``UPLOAD_CONFIG_FILE`` : Upload your existing Rclone config file. It will be placed in its correct location automatically.<br>``SAVE_TO_MYDRIVE`` : Save Rclone config file in Google Drive.<br>``DOWNLOAD_CONFIG`` : Download Rclone config file in your Device.**


In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center>
#@markdown <center><h3><b>Create/Edit/Delete Rclone Config File</b></h3></center><br>

from google.colab import files
from sys import exit as end
from pathlib import Path, PurePath

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(f'{CMP}').exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(DRIVE_CONFIG_DIR).exists():
  !rclone mkdir {DRIVE_CONFIG_DIR}
  print('Created: MCT-Config [folder] -> MyDrive')

#@markdown ****
UPLOAD_CONFIG_FILE = False #@param {type:"boolean"}
SAVE_TO_MYDRIVE = True #@param {type:"boolean"}
DOWNLOAD_CONFIG = False #@param {type:"boolean"}

if UPLOAD_CONFIG_FILE and SAVE_TO_MYDRIVE and DOWNLOAD_CONFIG:
  # Initiate Upload Request
  print('Request: Initiate upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path("/root/.config/rclone/").exists():
    !rclone mkdir "/root/.config/rclone/"
  !rclone copy "/content/rclone.conf" "/root/.config/rclone/"
  print('Saved: rclone.conf [file] -> Runtime')
  # Rclone configuration
  !rclone config
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
  # Initiate Download Request
  files.download("/root/.config/rclone/")
  print('Request: Initiated download -> rclone.conf [file]')
elif UPLOAD_CONFIG_FILE and SAVE_TO_MYDRIVE:
  # Initiate Upload Request
  print('Request: Initiate upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path('/root/.config/rclone/').exists():
    !rclone mkdir "/root/.config/rclone/"
  !rclone copy "/content/rclone.conf" "/root/.config/rclone/"
  print('Saved: rclone.conf [file] -> Runtime')
  # Rclone configuration
  !rclone config
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
elif UPLOAD_CONFIG_FILE:
  # Initiate Upload Request
  print('Request: Initiate Upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path('/root/.config/rclone/').exists():
    !rclone mkdir "/root/.config/rclone/"
  print('[WARNING] -> No saving option selected, progress may lose.')
  # Rclone configuration
  !rclone config
elif SAVE_TO_MYDRIVE:
  # Rclone configuration
  !rclone config
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
elif DOWNLOAD_CONFIG:
  # Rclone configuration
  !rclone config
  # Initiate Download Request
  files.download("/root/.config/rclone/")
  print('Request: Initiated download -> rclone.conf [file]')
else:
  print('[WARNING] -> No saving option selected, progress may lose.')
  !rclone config

**Mount Rclone remotes on this runtime:**

**Options:**<br>
`CUSTOM_MOUNT_POINT`**: Set custom mount point.**<br>
`CUSTOM_CACHE_DIR`**: Set custom cache directory.**

In [ ]:
# @markdown <br>
# @markdown <center><h3><b>Connect To Cloud Accounts</b></h3></center>
# @markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Multiple-Clouds.webp" height="100" alt="Multiple-Clouds"/></center>
# @markdown <center><b>Connect your Cloud Accounts (Rclone remotes) between which you want to transfer Data!</b></center>

from sys import exit as end
from os import makedirs, system
from pathlib import Path, PurePath
from re import findall, M

if not Path("/root/.config/rclone/rclone.conf").exists():
  end('Not Found: rclone.conf [file]')

content = open("/root/.config/rclone/rclone.conf").read()
avCon = findall(r"^\[(.+)\]$", content, M)

print("Available Rclone remotes:")
print(*avCon)
remote = input("Enter Rclone remote name to continue:\n")

if remote not in avCon:
  end('Not Found: Given remote name not exist in rclone.conf [file]')

# User Input
CUSTOM_MOUNT_POINT = False  # @param {type:"boolean"}
CUSTOM_CACHE_DIR = False  # @param {type:"boolean"}

# Custom Mount Point
if CUSTOM_MOUNT_POINT:
  Mount_Point = input("Enter Custom Mount path:\n")
  if not Mount_Point:
    end("Mount Path can't be empty.")
  elif Path(Mount_Point).exists() and Path(Mount_Point).is_dir():
      Mount_Point = f"{PurePath(Mount_Point)}/{remote}"
  elif not Path(Mount_Point).exists():
      end('Not Found: Given mount path not found.')
  elif not Path(Mount_Point).is_dir():
    end('Path Error: Given path is not a valid directory.')
else:
  Mount_Point = f"/content/drives/{remote}"

# Custom Cache Directory
if CUSTOM_CACHE_DIR:
  Cache_dir = input("Enter Custom Cache Directory path:\n")
  if not Cache_dir:
    end("Cache Directory Path can't be empty.")
  elif Path(Cache_dir).exists() and Path(Cache_dir).is_dir():
    Cache_dir = f"{PurePath(Cache_dir)}/{remote}"
  elif not Path(Cache_dir).exists():
    end('Not Found: Given mount path not found.')
  elif not Path(Cache_dir).exists():
    end('Path Error: Give path is not a valid directory.')
else:
  Cache_dir = "/content/temp/Rclone"

Action = input("Select Action: 1 = Mount and 0 = Unmount:\n")

if Action not in ['0','1']:
  end("Input Error: Value should be 1 or 0.")
elif Action == '1':
  makedirs(Mount_Point, exist_ok=True)
  !rclone mount {remote}: {Mount_Point} --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir {Cache_dir} --allow-other --daemon
  print(f'Mounted: {Mount_Point}') if Path(Mount_Point).exists() else print(f"Mount Error: Unable to mount {Mount_Point}")
elif Action == '0' and system(f"fusermount -uz {Mount_Point}") == 0:
  !rm -r {Mount_Point}
else:
  system(f"fusermount -uz {Mount_Point}")

**Using various Rclone modes, flags & other options:**



**Options:**<br>
`DRY_RUN`**: Check given source & destination.**<br>
`EMAIL_NOTIFICATION`**: Receive email notification once task is completed.**<br>
`LOGS`**: Send log file with email notification.**

In [ ]:
# @markdown <br>
# @markdown <center><img src="https://assets.thecaduceus.eu.org/MCT/img/rclone_logo_text.png" alt="rclone forum" width=150></center>

# @markdown ---
import smtplib
SOURCE = ""  # @param {type:"string"}
DESTINATION = ""  # @param {type:"string"}
MODE = "copy"  # @param ["copy", "copyto", "copyurl", "move", "moveto", "check", "sync", "bisync", "cat", "mkdir","cleanup", "ls", "lsd", "lsl", "test makefile","md5sum", "sha1sum", "size", "genautocomplete", "gendocs", "tree","obscure"]
DRY_RUN = False  # @param {type:"boolean"}
ARGUMENTS = ""  # @param {type:"string"}
# @markdown ---
EMAIL_NOTIFICATION = False  # @param {type:"boolean"}
LOGS = False  # @param {type:"boolean"}
EMAIL_ID = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
RECEIVER_ID = ""  # @param {type:"string"}
# @markdown ---

args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
if MODE == "check":
  args += "--one-way "
if DRY_RUN:
    args += " --dry-run "
if LOGS:
  args += " --log-file Rclone_log.txt "
if ARGUMENTS != "":
  args += ARGUMENTS

def runRclone():
  !rclone --user-agent "Mozilla" "$MODE" "$SOURCE" "$DESTINATION" $args


def checkEmail():
  if EMAIL_NOTIFICATION and not LOGS:
    print("You will receive only a notification after task has finished.")
    runRclone()
    s = _extracted_from_checkEmail_7(EMAIL_ID)
    message = "Your rClone task has Completed!"
    s.sendmail(EMAIL_ID, RECEIVER_ID, message)
    print("Email Alert Sent!")
    s.quit()
  elif EMAIL_NOTIFICATION and LOGS == True:
    print("You will receive a notification with log attached after task has finished.")
    runRclone()
    # Sending email notification with logs

    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders

    fromaddr = EMAIL_ID
    toaddr = RECEIVER_ID
    msg = MIMEMultipart()
    # storing the senders email address
    msg['From'] = fromaddr
    # storing the receivers email address
    msg['To'] = toaddr
    # storing the subject
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail
    body = "Your Rclone Task has Completed!"

    msg.attach(MIMEText(body, 'plain'))
    filename = "Rclone_log.txt"
    attachment = open(f"/content/{filename}", "rb")
    p = MIMEBase('application', 'octet-stream')
    p.set_payload((attachment).read())
    # encode into base64
    encoders.encode_base64(p)
# p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    p.add_header('Content-Disposition', f"attachment; filename= {filename}")
    msg.attach(p)
    s = _extracted_from_checkEmail_7(fromaddr)
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent with log!")
    s.quit()
  elif not EMAIL_NOTIFICATION and not LOGS:
    print("You will not receive any notification.")
    runRclone()

def _extracted_from_checkEmail_7(arg0):
  result = smtplib.SMTP('smtp.gmail.com', 587)
  result.starttls()
  result.login(arg0, PASSWORD)
  return result

checkEmail()

**Delete your Files/Folder/Directory from below:<br>``delete`` Delete the content of folder or single file.<br>``purge`` Delete whole directory.<br>``rmdir`` Completely delete the given path.<br>``rmdirs`` Remove any empty directories under the path.**

In [ ]:
from sys import exit as end
from pathlib import Path

# User Input
ACTION = "delete"  # @param ["delete", "purge", "rmdir", "rmdirs"]
PATH = ""  # @param {type:"string"}
ARGUMENTS = ""  # @param {type:"string"}

if not PATH:
    end("Path Error: Path can't be empty.")
elif Path(PATH).exists():
    !rclone {ACTION} "$PATH" $ARGUMENTS
    print('Execution Completed!')
elif not Path(PATH).exists():
    end("Path Error: Given path not exist.")
else:
    print('[ERROR] - Unhandled Action.')

####**Add-ons:**

In [ ]:
from libtorrent import session as lt_session, add_magnet_uri, torrent_info
from sys import exit as end, stdout
from httpx import post as post_url, get as get_url
from pathlib import Path, PurePath
from google.colab import files as colab_file
from os import makedirs, strerror
from json import load as json_load, dumps as dump_json
from subprocess import Popen, PIPE, STDOUT, run
from contextlib import closing
from functools import wraps
from errno import ETIME
from signal import signal, alarm, SIGALRM
from getpass import getpass
from time import time, sleep
import shlex

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
	!rclone mkdir "$DRIVE_CONFIG_DIR"
	print('Created: MCT-Config [folder] -> MyDrive')

# Retrive Json Data
if Path('mct.json').exists():
  with open('mct.json','r') as mct_json:
    json_data = json_load(mct_json)
else:
  print("[INFO] - Downloading mct.json [file]")
  json_data = get_url("https://assets.thecaduceus.eu.org/MCT/json/mct.json").json()
  with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

# User Input
SOURCE = ""  # @param {type:"string"}
MODE = "YouTube Videos/Playlist & Other Sites" # @param ["Direct Download Links", "Download Links from txt file", "YouTube Videos/Playlist & Other Sites", "Torrent DL", "Torrent DL (File Selection)", "Upload file to Anonymous Platforms","MEGA Link DL", "UUP DUMP"]
DESTINATION = ""  # @param {type:"string"}
args = ""  # @param {type:"string"}


def runSh(args, *, output=False, shell=False, cd=None):
  if shell:
    return ((run(
        args,
        shell=True,
        stdout=PIPE,
        stderr=STDOUT,
        cwd=cd,
    ).stdout.decode("utf-8").strip()) if output else run(
        args, shell=True, cwd=cd).returncode)
  if output:
    proc = Popen(shlex.split(args), stdout=PIPE, stderr=STDOUT, cwd=cd)
    while True:
      output = proc.stdout.readline()
      if output == b"" and proc.poll() is not None:
        return
      if output:
        print(output.decode("utf-8").strip())
  return run(shlex.split(args), cwd=cd).returncode

def mega_dl():
  global DESTINATION
  if not DESTINATION:
    makedirs("downloads", exist_ok=True)
    DESTINATION = "downloads"
  newlines = ['\n', '\r\n', '\r']
  def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with closing(stream):
      while True:
        out = []
        last = stream.read(1)
        if last == '' and proc.poll() is not None:
          break
        while last not in newlines:
          if last == '' and proc.poll() is not None:
            break
          out.append(last)
          last = stream.read(1)
        out = ''.join(out)
        yield out

  def transfer():
    cmd = ["mega-get", SOURCE, f"{DESTINATION}"]
    proc = Popen(
      cmd,
      stdout=PIPE,
      stderr=STDOUT,
      universal_newlines=True,
    )
    for line in unbuffered(proc):
      print(line)
  transfer()

def config_creds():
  class TimeoutError(Exception):
    pass

  def timeout(seconds=10, error_message=strerror(ETIME)):
    def decorator(func):
      def _handle_timeout(signum, frame):
        raise TimeoutError(error_message)

      def wrapper(*args, **kwargs):
        signal(SIGALRM, _handle_timeout)
        alarm(seconds)
        try:
          result = func(*args, **kwargs)
        finally:
          alarm(0)
        return result
      return wraps(func)(wrapper)
    return decorator

  @timeout(10)
  def runShT(args):
    return runSh(args, output=True)
  if not Path("/usr/bin/mega-cmd").exists():
    print("[ERROR] - MEGA not installed.")
  else:
    !pkill mega-cmd

  with open('mega.json', 'r') as mega_json:
    mega_json = json_load(mega_json)
    print('[INFO] - Decrypting MEGA credentials.')
  try:
    decrypted_mega_id = post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": f"{mega_json['mega_id']}"}).json()
    decrypted_mega_pass = post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": f"{mega_json['mega_pass']}"}).json()
  except Exception as error_code:
    end(f"Unable to decrypt MEGA Credentials.\nTraceback:\n{error_code}")

  try:
    runShT(f"mega-login {decrypted_mega_id['result']} {decrypted_mega_pass['result']}")
  except TimeoutError:
    runSh('mega-whoami', output=True)
  mega_dl()

def save_creds():
  mega_id = input('Enter MEGA Email ID or Username:\n')
  mega_pass = getpass('Enter MEGA Account Password: (Input is hidden)\n')
  if not mega_id:
    end("Input Error: MEGA ID can't be blank!")
  elif not mega_pass:
    end("Input Error: MEGA Password can't be blank!")
  elif len(mega_pass) < 8:
    end("Input Error: MEGA Password must be of 8 digit or greater.")

  print('[INFO] - Encrypting MEGA Credentials')
  try:
    encrypted_mega_id = post_url(f'{json_data["apis"]["encryptAPI"]}/encrypt', json={"string": f"{mega_id}"}).json()
    encrypted_mega_pass = post_url(f'{json_data["apis"]["encryptAPI"]}/encrypt', json={"string": f"{mega_pass}"}).json()
  except Exception as error_code:
    end(f"Unable to encrypt MEGA Credentials.\nTraceback:\n{error_code}")
  encrypted_mega_json = {"mega_id": f"{encrypted_mega_id['result']}", "mega_pass": f"{encrypted_mega_pass['result']}"}
  with open('mega.json','w') as save_json:
    save_json.write(dump_json(encrypted_mega_json))
    print('Created: mega.json [file]')
    save_json.close()
  !rclone copy "/content/mega.json" "$DRIVE_CONFIG_DIR"
  print('Saved: mega.json -> MyDrive')
  config_creds()

if MODE =="Direct Download Links":
  links = SOURCE.split(',')
  for link in links:
    %cd "$DESTINATION"
    !aria2c "$link" $args
  print('Execution completed!')
elif MODE == "Download Links from txt file":
  %cd "$DESTINATION"
  !aria2c -i "$SOURCE" $args
  print('Execution completed!')
elif MODE == "YouTube Videos/Playlist & Other Sites":
  %cd "$DESTINATION"
  !yt-dlp "$SOURCE" $args
  print('Execution completed!')
elif MODE =="Torrent DL":
  ses = lt_session()
  ses.listen_on(6881, 6891)
  user_input = input("Enter magnet link or torrent file path:\n") if not SOURCE else SOURCE
  if user_input.startswith("magnet:"):
    params = {"save_path": DESTINATION}
    handle = add_magnet_uri(ses, user_input, params)
  elif Path(user_input).exists():
    info = torrent_info(user_input)
    params = {"ti": info, "save_path": DESTINATION}
    handle = ses.add_torrent(params)
  else:
    end("Invalid input or given path not exist.")

  while not handle.status().is_seeding:
    s = handle.status()
    print(f"\r{s.progress * 100:.2f}% | DL: {s.download_rate / 1000000:.2f} MB/s | UP: {s.upload_rate / 1000000:.2f} MB/s | Peers: {s.num_peers} | Seeds: {s.num_seeds}", end=" ")
    stdout.flush()
    sleep(1)

elif MODE == "Torrent DL (File Selection)":
  ses = lt_session()
  ses.listen_on(6881, 6891)
  user_input = input("Enter magnet link or torrent file path:\n") if not SOURCE else SOURCE
  if user_input.startswith("magnet:"):
    params = {"save_path": DESTINATION}
    handle = add_magnet_uri(ses, user_input, params)
  elif Path(user_input).exists():
    info = torrent_info(user_input)
    params = {"ti": info, "save_path": DESTINATION}
    handle = ses.add_torrent(params)
  else:
    end("Invalid input or given path not exist.")

  print("Downloading metadata...")
  start_time = time()
  while not handle.has_metadata():
    elapsed_time = time() - start_time
    if elapsed_time > 30: # Increase timeout time from here, by default its 30 seconds
      end("Torrent timeout, failed to load metadata.")

  print("Metadata downloaded.")

  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  print("Files in torrent:")
  for i, f in enumerate(files):
    print(f"{i+1}: {f.path} ({f.size / 1000000:.2f} MB)")

  file_nums = input("Enter file numbers to download [separated by space or dash (for range)]:").split()

  priorities = [0] * len(files)
  for n in file_nums:
    if "-" in n:
      start_stop = n.split("-")
      start = int(start_stop[0]) - 1
      stop = int(start_stop[1])
      if start_stop[0] >= start_stop[1]:
        end("Incorrect range.")
      for i in range(start, stop):
        priorities[i] = 1
    else:
      priorities[int(n) - 1] = 1

  handle.prioritize_files(priorities)

  print("Downloading...")
  while not handle.is_seed():
    s = handle.status()
    print(f"\r{s.progress * 100:.2f}% | DL: {s.download_rate / 1000000:.2f} MB/s | UP: {s.upload_rate / 1000000:.2f} MB/s | Peers: {s.num_peers} | Seeds: {s.num_seeds}", end=" ")
    stdout.flush()
    sleep(1)
elif MODE == "Upload file to Anonymous Platforms":
  if not SOURCE:
    end("Source path can't be empty!")
  elif not Path(SOURCE).exists():
    end("Not Found: Given path not exists!")

  API = list(json_data['apis']['anonFileShare'].values())
  site_names = ""

  for num, name in enumerate(list(json_data['apis']['anonFileShare'].keys())):
    site_names += f"{num}.{name}\n"

  while True:
    count = int(input(f"Choose Platform by its number (0 - {len(API) - 1}):\n{site_names}>"))

    if count > len(API) - 1:
      print('Selection out of list.')
    else:
      print(f'Selection -> {count}\nAPI -> {API[count]}\n')
      break

  if not Path(SOURCE).exists():
    end('Not Found: Given source path not exist.')
  elif Path(SOURCE).is_dir():
    links = []
    is_recursive = input("Upload files recursively? 0 = No & 1 = Yes:\n")
    if is_recursive == '1':
      print("Scanning path for all possible directories...")
      for file_path in list(Path(SOURCE).rglob('**/*.*')):
      # This check is still important, since Google Colab secretly creates '.ipynb_checkpoints' detected as folder by pathlib
        if Path(file_path).is_file():
          try:
            print(f"Uploading [{PurePath(file_path).name}]")
            api_resp = post_url(API[count], files = {'file': open(file_path,'rb')}).json()
            print(f"{PurePath(file_path).name} -> " + api_resp["data"]["file"]["url"]["full"])
            links.append(f"{PurePath(file_path).name}:\n" + api_resp["data"]["file"]["url"]["full"])
          except OverflowError:
            print(f"File [{PurePath(file_path).name}] not allowed.")
          except KeyError as error_code:
            print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
          except Exception as error_code:
            print(f"An error occured!\nTraceback:\n{error_code}")
            sleep(4)
        else:
          print(f"[Dir] -> {PurePath(file_path).name}")
    else:
      for file_path in list(Path(SOURCE).rglob("*.*")):
        if Path(file_path).is_file():
          try:
            print(f"Uploading [{PurePath(file_path).name}]")
            api_resp = post_url(API[count], files = {'file': open(file_path,'rb')}).json()
            print(f"{PurePath(file_path).name} -> " + api_resp["data"]["file"]["url"]["full"])
            links.append(f"{PurePath(file_path).name}:\n" + api_resp["data"]["file"]["url"]["full"])
          except OverflowError:
            print(f"File [{PurePath(file_path).name}] not allowed.")
          except KeyError as error_code:
            print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
          except Exception as error_code:
            print(f"An error occured!\nTraceback:\n{error_code}")
            sleep(4)
        else:
          print(f"[Dir] -> {PurePath(file_path).name}")

    if len(links) > 1:
      print(f"Total Links -> {len(links)}")
      is_export = input("Do you want to export all links in txt file? 0 = No or 1 = Yes:\n")
      if is_export not in ['0','1'] or is_export == '1':
        with open('links.txt','a+') as save_links:
          save_links.writelines('%s\n' % item for item in links)
          save_links.close()
          colab_file.download('links.txt')
          print('Request: Initiated download -> links.txt [file]')
      else:
        with open('links.txt','w+') as save_links:
          save_links.ritelines('%s\n' % item for item in links)
          save_links.close()
          print("Saved: links.txt [file] -> Runtime")
    else:
      print(f'Total Links -> {len(links)}')
  elif Path(SOURCE).is_file():
    try:
      print(f"Uploading [{PurePath(SOURCE).name}]")
      api_resp = post_url(API[count], files={'file': open(SOURCE, 'rb')}).json()
      print(f"{PurePath(SOURCE).name} -> " + api_resp["data"]["file"]["url"]["full"])
    except OverflowError:
      print("File not allowed.")
    except Exception as error_code:
      print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")

elif MODE =="MEGA Link DL":
  use_account = input('Do you want to use your MEGA Account? to lift limits imposed on anonymous downloader? 0 = No & 1 = Yes:\n')
  if use_account not in ['0','1']:
    end("Input Error: Invalid input parameter! Please enter 0 or 1 as input.")
  elif use_account == '1' and Path('/content/mega.json').exists():
    use_existing = input('[INFO] - Found existing MEGA Account credentials, do you want to use them? 0 = No & 1 = Yes:\n')
    if use_existing not in ['0','1']:
      end("Input Error: Invalid input parameter! Please enter 0 or 1 as input.")
    elif use_existing == '1':
      config_creds()
    elif use_existing == '0':
      save_creds()
  elif use_account == '1':
    save_creds()
  else:
    mega_dl()

elif MODE == "UUP DUMP":
  if Path(SOURCE).is_file() and PurePath(SOURCE).name == 'uup_download_linux.sh':
    !bash "$SOURCE"
  elif Path(SOURCE).is_dir() and Path(f"{SOURCE}/uup_download_linux.sh").exists():
    !bash "$SOURCE/uup_download_linux.sh"
  else:
    print("Not Found: uup_download_linux.sh [file]")

####**Packaging & Extraction:**

In [ ]:
from sys import exit as end
from pathlib import Path, PurePath
from getpass import getpass

# User Input
ACTION = "Zip" # @param ["Zip", "Zip with Password", "RaR", "RaR with Password", "7z", "7z with Password", "Tar", "Tar.gz", "Tar.bz2", "UnZip", "UnZip with Password", "UnRaR", "UnRaR with Password", "Extract 7z", "Extract 7z with Password", "Extract Tar", "Extract Tar.gz", "Extract Tar.bz2"]
SOURCE_PATH = "" #@param {type:"string"}
OUTPUT_FILENAME = "" #@param {type:"string"}
DESTINATION_PATH = "" #@param {type:"string"}
args = "" #@param {type:"string"}
ENCRYPT_FILENAME = False  # @param {type:"boolean"}

# Check for Setup & common things
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not ACTION:
  end("Select a supported operation to perform.")
elif not SOURCE_PATH:
  end("Source field can't be empty.")
elif not DESTINATION_PATH:
  end("Destination field can't be empty")
elif not Path(SOURCE_PATH).exists():
  end("Not found: Given source path not exist.")
elif not Path(DESTINATION_PATH).exists():
  end("Not found: Given destination path not exist.")
else:
  SOURCE_PATH = PurePath(SOURCE_PATH)
  DESTINATION_PATH = PurePath(DESTINATION_PATH)
  OUTPUT_FILENAME = PurePath(SOURCE_PATH).name if not OUTPUT_FILENAME else OUTPUT_FILENAME

if ENCRYPT_FILENAME and not ACTION.endswith("with Password") or ACTION == "Zip with Password":
  print("[WARN] - Selected action not support ENCRYPT_FILENAME, continuing without it.")

if ACTION == "Zip":
  Pack_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.zip"
  !zip -r "$Pack_Dest" "$SOURCE_PATH" $args
elif ACTION == "Zip with Password":
  Pack_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.zip"
  password = getpass("Enter password:(Input is hidden)\n")
  !zip -P "$password" -r "$Pack_Dest" "$SOURCE_PATH" $args
elif ACTION == "UnZip":
  !unzip "$SOURCE_PATH" -d "$DESTINATION_PATH" $args
elif ACTION == "UnZip with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !unzip -P "$password" -d "$DESTINATION_PATH" "$SOURCE_PATH" $args
elif ACTION == "RaR":
  Rar_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.rar"
  !rar a "$Rar_Dest" "$SOURCE_PATH" $args
elif ACTION == "RaR with Password":
  Rar_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.rar"
  password = getpass("Enter password:(Input is hidden)\n")
  if ENCRYPT_FILENAME:
    !rar a -p"$password" -hp "$Rar_Dest" "$SOURCE_PATH" $args
  else:
    !rar a -p"$password" "$Rar_Dest" "$SOURCE_PATH" $args
elif ACTION == "UnRaR":
  !unrar x "$SOURCE_PATH" "$DESTINATION_PATH" $args
elif ACTION == "UnRaR with Password":
  !unrar x -Y "$SOURCE_PATH" "$DESTINATION_PATH" $args
elif ACTION == "7z":
  sz_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.7z"
  !7z a "$sz_Dest" "$SOURCE_PATH" $args
elif ACTION == "7z with Password":
  sz_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.7z"
  password = getpass("Enter password:(Input is hidden)\n")
  if ENCRYPT_FILENAME:
    !7z a -p"$password" -mhe=on "$sz_Dest" "$SOURCE_PATH" $args
  else:
    !7z a -p"$password" "$sz_Dest" "$SOURCE_PATH" $args
elif ACTION == "Extract 7z":
  !7z x "$SOURCE_PATH" -o"$DESTINATION_PATH" $args
elif ACTION == "Extract 7z with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !7z x -p"$password" -o"$DESTINATION_PATH" "$SOURCE_PATH" $args
elif ACTION == "Tar":
  Tar_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.tar"
  !tar -cvf "$Tar_Dest" "$SOURCE_PATH" $args
elif ACTION == "Extract Tar" :
  !tar -xvf "$SOURCE_PATH" -C "$DESTINATION_PATH" $args
elif ACTION == "Tar.gz":
  Targz_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.tar.gz"
  !tar -czvf "$Targz_Dest" "$SOURCE_PATH" $args
elif ACTION == "Extract Tar.gz":
  !tar -xzvf "$SOURCE_PATH" -C "$DESTINATION_PATH" $args
elif ACTION == "Tar.bz2":
  !tar -cjvf "$Tarbz_Dest" "$SOURCE_PATH" $args
elif ACTION == "Extract Tar.bz2":
  Tarbz_Dest = f"{DESTINATION_PATH}/{OUTPUT_FILENAME}.tar.bz"
  !tar -xjvf "$SOURCE_PATH" -C "$DESTINATION_PATH" $args
else:
  print("Invalid Operation.")

####**Telegram Integration:**

**Options:**<br>
`RECURSIVE`**: Upload files recursively from folder.**<br>

In [ ]:
#@markdown <br><center><img src="https://pnggrid.com/wp-content/uploads/2021/04/telegram-logo-circle-1536x1536.png" height="60">
#@markdown <center><h3><b><i>Telegram Transfer</i></b></h3>

from pathlib import Path, PurePath
from sys import exit as end
from json import load as json_load, dumps as dump_json
from httpx import get as get_url, post as post_url
from google.colab import output

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
	!rclone mkdir "$DRIVE_CONFIG_DIR"
	print('Created: MCT-Config [folder] -> MyDrive')

# Retrive Json Data
if Path('mct.json').exists():
  with open('mct.json', 'r') as mct_json:
    json_data = json_load(mct_json)
else:
  print("[INFO] - Downloading mct.json [file]")
  json_data = get_url("https://assets.thecaduceus.eu.org/MCT/json/mct.json").json()
  with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

# Load configuration
if Path('telegram.json').exists():
  with open('telegram.json', 'r') as conf_file:
    tg_json = json_load(conf_file)
  print("[INFO] - Decrypting API ID & API HASH.")
else:
  with open('telegram.json', 'w+') as conf_file:
    tg_json = {
        "version": "1.0",
        "tokens": {},
    }
    conf_file.write(dump_json(tg_json))

# Decrypt tokens
_API_ID = []
_API_HASH = []

for API_ID,API_HASH in zip([token.get("API_ID") for token in tg_json["tokens"].values()],[token.get("API_HASH") for token in tg_json["tokens"].values()]):
  _API_ID.append(int(post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": API_ID}).json()["result"]))
  _API_HASH.append(post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": API_HASH}).json()["result"])

# API Profile String
api_string = ""

for num, name in enumerate(list(tg_json["tokens"].keys()), start=1):
  api_string += f"{num}.{name}\n"

# User Profile String
user_string = ""

for num,name in enumerate(list(Path().glob("*.session")), start=1):
  user_string += f"{num}.{PurePath(name).name.split('.')[0]}\n" # name.split('.')[0] will not work

METHOD = "Manage API Profiles" #@param ["Manage API Profiles", "Manage Telegram Profiles", "Upload Files/Folder"]
SOURCE = "" #@param {type:"string"}
DESTINATION = "" #@param {type:"string"}
ARGUMENTS = "" #@param {type:"string"}
RECURSIVE = False #@param {type:"boolean"}

if METHOD == "Manage API Profiles":
  print(f"Availabe API Profiles: {len(_API_ID)}\n{api_string}")
  while True:
    action = input("(a)add/(r)remove/(e)exit>").lower()
    if action in ['a', 'add', '+']:
      api_profile_name = input("Give a name to this API profile:\n>")
      api_id = int(input("Enter your API ID:\n"))
      api_hash = input("Enter your API HASH:\n")
      print("[INFO] - Encrypting API ID & API HASH.")
      api_id = post_url(f'{json_data["apis"]["encryptAPI"]}/encrypt', json={"string": str(api_id)}).json()["result"]
      api_hash = post_url(f'{json_data["apis"]["encryptAPI"]}/encrypt', json={"string": api_hash}).json()["result"]
      tg_json["tokens"][api_profile_name] = {"API_ID": api_id, "API_HASH": api_hash}
      with open("telegram.json", "w+") as conf_file:
        conf_file.write(dump_json(tg_json))
        print(f"Saved [{api_profile_name}].")
      !rclone copy "telegram.json" "$DRIVE_CONFIG_DIR"
    elif action in ['r', 'remove', 'rm', '-']:
      api_profile_name = input("Enter API profile name you want to delete:\n")
      try:
        del tg_json['tokens'][api_profile_name]
        with open("telegram.json", "w+") as conf_file:
          conf_file.write(dump_json(tg_json))
          print(f"Deleted [{api_profile_name}].")
        !rclone copy "telegram.json" "$DRIVE_CONFIG_DIR"
      except KeyError:
        print("Given API profile not exist.")
    elif action in ['e','exit']:
      output.clear()
      break
elif METHOD == "Manage Telegram Profiles":
  print(f"Available User Profiles: {len(list(Path().glob('*.session')))}\n{user_string}")
  while True:
    action = input("(a)add/(r)remove/(e)exit>").lower()
    if action in ['a', 'add', '+']:
      if len(_API_ID) == 0:
        exit("Error: No API Profile configured for selection.")
      user_profile_name = input("Give a name to new Telegram profile:\n>")
      count = int(input(f"\nSelect API profile (use index number) to create new session with (1 - {len(_API_ID) if len(_API_ID) >= 1 else 1 - 1}):\n{api_string}\n>")) - 1
      try:
        api_id = _API_ID[count]
        api_hash = _API_HASH[count]
      except IndexError:
        exit("Selection out of list.")
      profile_type = input("Sign up as user or bot?\n>").lower()
      if profile_type == "user":
        phone_number = input("Enter your phone number in international format without '+' sign:\n>")
        print("Starting Telegram client as user...\n")
        !python tg-upload.py --profile "$user_profile_name" --api_id "$api_id" --api_hash "$api_hash" --phone "$phone_number" --login_only $ARGUMENTS
      else:
        bot_token = input("Enter your bot token:\n>")
        print("Starting Telegram client as bot...\n")
        !python tg-upload.py --profile "$user_profile_name" --api_id "$api_id" --api_hash "$api_hash" --bot "$bot_token" --login_only $ARGUMENTS

      session_path = f"{user_profile_name}.session"
      if Path(session_path).exists():
        !rclone copy "$session_path" "$DRIVE_CONFIG_DIR"
        print(f"Saved [{user_profile_name}].")
    elif action in ['r','remove','-']:
      user_profile_name = input(f"Enter name of Telegram profile you want to delete:\n>")

      if not Path(f"{user_profile_name}.session").exists() and not Path(f"{DRIVE_CONFIG_DIR}{user_profile_name}.session").exists():
        exit("Given Telegram profile not exist.")
      Path(f"{user_profile_name}.session").unlink(missing_ok=True)
      Path(f"{DRIVE_CONFIG_DIR}{user_profile_name}.session").unlink(missing_ok=True)
      print(f"Deleted [{user_profile_name}].")
    elif action in ['e','exit']:
      output.clear()
      break
elif METHOD == "Upload Files/Folder":
  if not SOURCE:
    exit("Source file/folder path can't be empty.")
  elif not DESTINATION:
    print("[NOTICE] - No destination chat identity passed, file will be uploaded to Saved messages.")
    DESTINATION = "me"

  if RECURSIVE:
    ARGUMENTS += " -r"

  profile_list = []
  for profile in list(Path().glob("*.session")):
    profile_list.append(Path(profile).name.split('.')[0])
  profile = int(input(f"Choose Telegram profile (use index number) to upload data using:\n{user_string}\n>")) - 1
  profile = profile_list[profile]
  !python tg-upload.py --profile "$profile" --path "$SOURCE" --chat_id "$DESTINATION" $ARGUMENTS

####**GoFile Integration:**

**Options:**<br>
`RECURSIVE`**: Upload files recursively from folder.**<br>
`RAW`**: Show API Response as raw.**

In [ ]:
# @markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/GoFile_log.png" height="50" alt="GoFile-Logo"/></center>
# @markdown <center><h3><i><b>GoFile Transfer</b></i></h3></center><br>

from httpx import get as get_url, post as post_url
from sys import exit as end
from pathlib import Path, PurePath
from json import load as json_load, dumps as dump_json
from ping3 import ping
from time import sleep
from google.colab import output
from google.colab.files import download as dl
from getpass import getpass
from socket import gethostbyname

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
	!rclone mkdir "$DRIVE_CONFIG_DIR"
	print('Created: MCT-Config [folder] -> MyDrive')

# Retrive Json Data
if Path('mct.json').exists():
  with open('mct.json', 'r') as mct_json:
    json_data = json_load(mct_json)
else:
  print("[INFO] - Downloading mct.json [file]")
  json_data = get_url("https://assets.thecaduceus.eu.org/MCT/json/mct.json").json()
  with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

METHOD = "Manage Tokens" # @param ["Server Details", "Upload Files", "Manage Tokens", "Account Details"]
SOURCE = ""  # @param {type:"string"}
DESTINATION = ""  # @param {type:"string"}
RECURSIVE = False  # @param {type:"boolean"}
RAW = False  # @param {type:"boolean"}

# Get Fastest Server
try:
  server_id = get_url(json_data["apis"]["GoFile"]["getServer"]).json()["data"]["server"]
except Exception as error_code:
  end(f"An error occured!\nTraceback:\n{error_code}")

# Load configuration
if Path('gofile.json').exists():
  with open('gofile.json', 'r') as conf_file:
    gofile_json = json_load(conf_file)
  print("[INFO] - Decrypting tokens.")
else:
  with open('gofile.json', 'w+') as conf_file:
    gofile_json = {
        "version": "2.0",
        "accounts": {}
    }
    conf_file.write(dump_json(gofile_json))

if gofile_json["version"] == json_data["apis"]["GoFile"]["configVersion"]:
  print(f"[INFO] - GoFile config version: [{gofile_json['version']}]")
elif gofile_json["version"] == "1.0":
  print(f"[UPGRADE] - GoFile config file version [1.0 -> {json_data['apis']['GoFile']['configVersion']}]")
  # Retrive data
  account_names = list(gofile_json["tokens"].keys())
  account_tokens = list(gofile_json["tokens"].values())
  # Decrypt tokens
  tokens = []
  for token in account_tokens:
    resp = post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": token}).json()["result"]
    tokens.append(resp)
  # Retrive root folder ids
  rootFolder = []
  for token in tokens:
    resp = get_url(json_data['apis']['GoFile']['getAccountDetails'].format(token=token)).json()["data"]["rootFolder"]
    rootFolder.append(resp)
  # Arrange Json
  gofile_json = {"version": "2.0", "accounts": {}}
  for name, token, root in zip(account_names, account_tokens, rootFolder):
    gofile_json["accounts"][name] = {"token": token, "rootFolder": root}
  # Save Json
  with open('gofile.json', 'w+') as conf_file:
    conf_file.write(dump_json(gofile_json))
else:
  end(f"Unsupported GoFile config file version: {gofile_json['version']}")
output.clear()
tokens = []
for i in [account.get("token") for account in gofile_json["accounts"].values()]:
  tokens.append(post_url(f'{json_data["apis"]["encryptAPI"]}/decrypt', json={"string": i}).json()["result"])

profile_string = ""
for num, name in enumerate(list(gofile_json["accounts"].keys()), start=1):
  profile_string += f"{num}.{name}\n"

if METHOD == 'Server Details':
  print(f"Server Details:\nServer ID -> {server_id}\nServer URL -> {json_data['apis']['GoFile']['serverURL'].format(server_id=server_id)}\nServer IP -> {gethostbyname(json_data['apis']['GoFile']['serverURL'].format(server_id=server_id))}\nPing -> {round(ping(json_data['apis']['GoFile']['serverURL'].format(server_id=server_id)) * 1000, 2)} ms")
elif METHOD == 'Upload Files':
  while True:
    count = int(input(f"Choose Profile by its number (0 - {len(tokens) if len(tokens) >= 1 else 1 - 1}):\n0.Anonymous Upload\n{profile_string}>")) - 1
    if count > len(tokens) - 1:
      print("Selection out of list.")
    else:
      break
  output.clear()
  links = []
  if count == -1:  # For anonymous upload
    if not Path(SOURCE).exists():
      end("Not Found: Given source path not exist!")
    elif Path(SOURCE).is_dir():
      if RECURSIVE:
        print("Scanning path for all possible directories...")
        for file_path in list(Path(SOURCE).rglob('**/*.*')):
          # This check is still important, since Google Colab secretly creates '.ipynb_checkpoints' detected as folder by pathlib
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except OverflowError:
              print(f"File [{PurePath(file_path).name}] not allowed.")
            except KeyError as error_code:
              print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
            except Exception as error_code:
              print(f"An error occured!\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
      else:
        for file_path in list(Path(SOURCE).rglob("*.*")):
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except OverflowError:
              print(f"File [{PurePath(file_path).name}] not allowed.")
            except KeyError as error_code:
              print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
            except Exception as error_code:
              print(f"An error occured!\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
    elif Path(SOURCE).is_file():
      try:
        print(f"Uploading [{PurePath(SOURCE).name}]")
        api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(SOURCE, 'rb')}).json()
        if not RAW:
          print(f"{PurePath(SOURCE).name} -> " + api_resp['data']['downloadPage'])
        else:
          print(f"{PurePath(SOURCE).name}:")
          print(api_resp)
      except OverflowError:
        print("File is not allowed.")
      except Exception as error_code:
        print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
  else:  # For upload with specific token & folder
    account_list = list(gofile_json["accounts"].keys())
    DESTINATION = gofile_json["accounts"][account_list[count]]["rootFolder"] if not DESTINATION else DESTINATION
    if not Path(SOURCE).exists():
      end("Not Found: Given source path not exist.")
    elif Path(SOURCE).is_dir():
      if RECURSIVE:
        print("Scanning path for all possible directories...")
        for file_path in list(Path(SOURCE).rglob('**/*.*')):
          # This check is still important, since Google Colab secretly creates '.ipynb_checkpoints' detected as folder by pathlib
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}, data={'token': tokens[count], 'folderId': DESTINATION}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except OverflowError:
              print(f"File [{PurePath(file_path).name}] not allowed.")
            except KeyError as error_code:
              print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
            except Exception as error_code:
              print(f"An error occured!\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
      else:
        for file_path in list(Path(SOURCE).rglob("*.*")):
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}, data={'token': tokens[count], 'folderId': DESTINATION}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except OverflowError:
              print(f"File [{PurePath(file_path).name}] not allowed.")
            except KeyError:
              print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
            except Exception as error_code:
              print(f"An error occured!\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
    elif Path(SOURCE).is_file():
      print(f"Uploading [{PurePath(SOURCE).name}]")
      try:
        api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(SOURCE, 'rb')}, data={'token': tokens[count], 'folderId': DESTINATION}).json()
        if not RAW:
          print(f"{PurePath(SOURCE).name} -> " + api_resp['data']['downloadPage'])
        else:
          print(f"{PurePath(SOURCE).name}:")
          print(api_resp)
      except OverflowError:
        print(f"File [{PurePath(SOURCE).name}] not allowed.")
      except KeyError as error_code:
        print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
      except Exception as error_code:
        print(f"An error occured!\nTraceback:\n{error_code}")

  if len(links) > 1:
    print(f"Total Links -> {len(links)}")
    is_export = input("Do you want to export all links in txt file? 0 = No or 1 = Yes:\n")
    if is_export not in ['0', '1'] or is_export == '1':
      with open('gofile-links.txt', 'a+') as save_links:
        save_links.writelines('%s\n' % item for item in links)
        save_links.close()
      dl('gofile-links.txt')
      print('Request: Initiated download -> gofile-links.txt [file]')
    else:
      with open('gofile-links.txt', 'w+') as save_links:
        save_links.writelines('%s\n' % item for item in links)
        save_links.close()
      print("Saved: gofile-links.txt [file] -> Runtime")

elif METHOD == 'Manage Tokens':
  print(f"Availabe tokens: {len(tokens)}\n{profile_string}")
  while True:
    action = input("(a)add/(r)remove/(e)exit>").lower()
    if action in ['a', 'add', '+']:
      token_name = input("Give a name to your token:\n>")
      token_value = getpass("Enter your token value: (input is hidden)\n")
      print("[INFO] - Checking token.")
      try:
        resp = get_url(json_data['apis']['GoFile']['getAccountDetails'].format(token=token_value)).json()
        rootfolderid = resp["data"]["rootFolder"]
      except Exception as error_code:
        end(f"Unable to retrive information:\nAPI Response:\n{resp}\nTracback:\n{error_code}")
      print("[INFO] - Encrypting token.")
      token_value = post_url(f'{json_data["apis"]["encryptAPI"]}/encrypt', json={"string": token_value}).json()["result"]
      gofile_json["accounts"][token_name] = {"token": token_value, "rootFolder": rootfolderid}
      with open('gofile.json', 'w+') as conf_file:
        conf_file.write(dump_json(gofile_json))
        print(f"Saved [{token_name}].")
      !rclone copy "gofile.json" "$DRIVE_CONFIG_DIR"
    elif action in ['r', 'remove', 'rm', '-']:
      token_name = input("Enter name of token you want to delete:\n>")
      try:
        del gofile_json['accounts'][token_name]
        with open('gofile.json', 'w+') as conf_file:
          conf_file.write(dump_json(gofile_json))
          print(f"Deleted [{token_name}].")
          !rclone copy "gofile.json" "$DRIVE_CONFIG_DIR"
      except KeyError:
        print("Given token name not exist!")
    elif action in ['e','exit']:
      output.clear()
      break
elif METHOD == 'Account Details':
  while True:
    count = int(input(f"Choose Profile by its number (1 - {len(tokens) if len(tokens) >= 1 else 1 - 1}):\n{profile_string}>")) - 1
    if count > len(tokens) - 1:
      print("Selection out of list.")
    else:
      break

  output.clear()

  try:
    api_resp = get_url(json_data['apis']['GoFile']['getAccountDetails'].format(token=tokens[count])).json()
    if not RAW:
      print(f"Token:\n{api_resp['data']['token']}\nLogin URL:\n" + json_data['apis']['GoFile']['loginURL'].format(token=api_resp['data']['token']) + f"\nEmail:\n{api_resp['data']['email']}\nTier:\n{api_resp['data']['tier']}\nCredit:\n{api_resp['data']['currencySign']}{api_resp['data']['credit']}\nRoot Folder ID:\n{api_resp['data']['rootFolder']}\nTotal Size:\n{api_resp['data']['totalSize'] / 1000000:.2f} MB\nTotal Files:\n{api_resp['data']['filesCount']}\nTotal Downloads:\n{api_resp['data']['total30DDLTraffic']}")
    else:
      print(api_resp)
  except KeyError as error_code:
    print(f"Key mapping error! printing json as raw:\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
  except Exception as error_code:
    print(f"An error occured!\n{error_code}")

#### **Run Commands & Code:**

**You can run any supported command below after
downloading its setup from above!<br>List of Commands:**
```
# Putting '!' before any command is mandatory!
---
!rclone [ACTION] [FLAGS]
---
!yt-dlp [URL-1 URL-2 URL-3] [OPTIONS]
---
!aria2c <torrent/magnet/url>
```

**Write your Code Below:**

**Code 1**

In [ ]:
  #<--Write Code before #

**Code 2**

In [ ]:
  #<--Write Code before #